In [2]:
import pandas as pd
import json
import requests
import time
import datetime



In [3]:
baseurl = 'https://api.usa.gov/crime/fbi/sapi/'
agencyapi = 'api/agencies'

params = {'api_key' : "qJDHVz0Vl9eSWuvXnvQ9HxHQPNeVRYdw08xTlUfw"}


counties = pd.read_csv('bureau_of_labor_counties_with_data.csv')
counties['cname'] = counties['County Name'].str.replace(' County', '').str.upper()
counties.head()

,Area Type,Area Code,County Name,State,Display Level,Selectable,Sort Sequence,cname
0,F,CN0108100000000,Lee County,AL,0,T,71,LEE
1,F,CN0100700000000,Bibb County,AL,0,T,34,BIBB
2,F,CN0102700000000,Clay County,AL,0,T,44,CLAY
3,F,CN0104500000000,Dale County,AL,0,T,53,DALE
4,F,CN0106500000000,Hale County,AL,0,T,63,HALE


In [4]:
oris = requests.get(baseurl + agencyapi, params=params).json() #Retrieve all agencies from source
# print(json.dumps(oris, indent=2, sort_keys=True))

In [5]:
# crimedata = requests.get(baseurl + '/api/summarized/agencies/AK0010200/offenses', params)
# crimedata
# mydat =

In [6]:
# crimedata.json()

In [10]:
mydata = {'Agency':[], 'Year':[], 'Offense':[],'count':[], 'county':[], 'state':[]}
print(f"starting at {datetime.datetime.now()}")
x = 0
for mystate in oris:
    if x < 25:
        x+=1
        print('do nothing')
    else:
        print(mystate)
    #     print(counties[counties['State'] == mystate]['State'].count())
        if counties[counties['State'] == mystate]['State'].count() > 0: #Check to see if state in county list
            statecounties = counties[counties['State'] == mystate]
    #         print(mystate)
        #     print(oris[mystate])
            for myori in oris[mystate]:

    #             print(statecounties[statecounties['cname'] ==  oris[mystate][myori]['county_name']])
                if  statecounties[statecounties['cname'] ==  oris[mystate][myori]['county_name']]['cname'].count() > 0: #Check to see if county in list
                    print('     ' + oris[mystate][myori]['county_name'])
                    crimedata = ''        
                    while crimedata == '':
                        try:
                            oriapi = f'/api/summarized/agencies/{myori}/offenses'
                            crimedata = requests.get(baseurl + oriapi, params).json()
                            crimedata['results']
                            for result in crimedata['results']:
                                if  2010 <= result['data_year']  <= 2015:
                                    mydata['Agency'].append(myori)
                                    mydata['Year'].append(result['data_year'])
                                    mydata['Offense'].append(result['offense'])
                                    mydata['count'].append(result['actual'])
                                    mydata['county'].append(oris[mystate][myori]['county_name'])
                                    mydata['state'].append(mystate)
                        except KeyError:
                            crimedate = ''
                            print('waiting 15 minutes')
                            print(datetime.datetime.now())
                            time.sleep(901)
                            if params['api_key'] == "qJDHVz0Vl9eSWuvXnvQ9HxHQPNeVRYdw08xTlUfw":
                                params['api_key'] = "yFiT76PCFjhbEBzdI7Wd2Z6bw1byGDjv7K6zWouf"
                            else:
                                params['api_key'] = "qJDHVz0Vl9eSWuvXnvQ9HxHQPNeVRYdw08xTlUfw"



        
            
        
        

starting at 2018-11-05 09:36:20.126725
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
do nothing
NE
     NUCKOLLS
waiting 15 minutes
2018-11-05 09:36:20.679979
     MORRILL
     THAYER
     SCOTTS BLUFF
     DAKOTA
     BOONE
     JOHNSON
     RED WILLOW
     MADISON
     FILLMORE
     HOWARD
     ADAMS
     DAWSON
     SHERIDAN
     SALINE
     RICHARDSON
     KEARNEY
     SEWARD
     ANTELOPE
     YORK
     HAMILTON
     CHEYENNE
     VALLEY
     HOLT
     KIMBALL
     MCPHERSON
     DIXON
     GARFIELD
     MERRICK
     BUFFALO
     CASS
     KEITH
     CEDAR
     LOGAN
     STANTON
     MERRICK
     SCOTTS BLUFF
     PHELPS
     CASS
     RED WILLOW
     LINCOLN
     PHELPS
     CUSTER
     BOYD
     DOUGLAS
     HOOKER
     DAWES
     GOSPER
     SHERIDAN
     DUNDY
 

     SUSSEX
     ATLANTIC
     BURLINGTON
     MONMOUTH
     MONMOUTH
     ESSEX
     CAMDEN
     HUDSON
     CAPE MAY
     BERGEN
     BERGEN
     BERGEN
     OCEAN
     SALEM
     GLOUCESTER
     CAMDEN
     MONMOUTH
     MORRIS
     MERCER
     PASSAIC
     ATLANTIC
     ESSEX
     BURLINGTON
     SOMERSET
     HUDSON
     ESSEX
     ESSEX
     BERGEN
     BERGEN
     OCEAN
     OCEAN
     SALEM
     BERGEN
     CAMDEN
     GLOUCESTER
     GLOUCESTER
     MORRIS
     PASSAIC
     ATLANTIC
     UNION
     BURLINGTON
     SOMERSET
     MIDDLESEX
     BERGEN
     HUNTERDON
     CAPE MAY
     BERGEN
     SALEM
     WARREN
     GLOUCESTER
     GLOUCESTER
     CAMDEN
     BURLINGTON
     MONMOUTH
     ATLANTIC
     PASSAIC
     MERCER
     ESSEX
     MONMOUTH
     BURLINGTON
     CAMDEN
     HUDSON
     ESSEX
     BERGEN
     BERGEN
     HUNTERDON
     BERGEN
     SUSSEX
     MONMOUTH
     GLOUCESTER
     CAMDEN
     BURLINGTON
     BURLINGTON
     MONMOUTH
     ATLANTIC
     MERCER
     

     CALEDONIA
     CHITTENDEN
     ORLEANS
     CHITTENDEN
     BENNINGTON
     BENNINGTON
     FRANKLIN
     FRANKLIN
     BENNINGTON
NM
     MCKINLEY
     QUAY
     SAN MIGUEL
     HIDALGO
     COLFAX
     EDDY
     TAOS
     SAN JUAN
     DONA ANA
     CIBOLA
     CHAVES
     DONA ANA
     DE BACA
     SANDOVAL
     SOCORRO
     LINCOLN
     GUADALUPE
     LOS ALAMOS
     VALENCIA
     VALENCIA
     OTERO
     TORRANCE
     LEA
     SIERRA
     ROOSEVELT
     LEA
     GRANT
     GRANT
     UNION
     COLFAX
     SAN MIGUEL
     EDDY
     SAN JUAN
     VALENCIA
     TAOS
     TAOS
     CHAVES
     CIBOLA
     LUNA
     DONA ANA
     QUAY
     DONA ANA
     UNION
     SANDOVAL
     BERNALILLO
     SOCORRO
     LINCOLN
     LINCOLN
     OTERO
     SANDOVAL
     TORRANCE
     LEA
     SIERRA
     ROOSEVELT
     GRANT
     CURRY
     SAN MIGUEL
     EDDY
     RIO ARRIBA
     COLFAX
     EDDY
     TAOS
     SAN JUAN
     HARDING
     CHAVES
     CHAVES
     LUNA
     QUAY
     DONA ANA
 

     OSCEOLA
     ORANGE
     ORANGE
     SARASOTA
     MADISON
     BROWARD
     GADSDEN
     UNION
     HILLSBOROUGH
     PINELLAS
     MIAMI-DADE
     OKEECHOBEE
     MARION
     BROWARD
     MIAMI-DADE
     BRADFORD
     SEMINOLE
     PALM BEACH
     MARION
     LAKE
     MIAMI-DADE
     CALHOUN
     BREVARD
     MARTIN
     LAFAYETTE
     MONROE
     HILLSBOROUGH
     FRANKLIN
     BROWARD
     PALM BEACH
     SARASOTA
     BROWARD
     ALACHUA
     DIXIE
     BROWARD
     MIAMI-DADE
     LAKE
     CALHOUN
     HIGHLANDS
     BREVARD
     ORANGE
     MIAMI-DADE
     ESCAMBIA
     LAFAYETTE
     LEE
     CALHOUN
     HERNANDO
     MARTIN
     GADSDEN
     ORANGE
     PALM BEACH
     SEMINOLE
     LAKE
     MIAMI-DADE
     ORANGE
     MIAMI-DADE
     SUWANNEE
     HILLSBOROUGH
     MARION
     CALHOUN
     MARTIN
     DUVAL
     DIXIE
     LAFAYETTE
     CHARLOTTE
     SARASOTA
     COLUMBIA
     HILLSBOROUGH
     DESOTO
     BROWARD
     PINELLAS
     SANTA ROSA
     LAKE
     MIAM

     WYOMING
     SARATOGA
     CORTLAND
     TOMPKINS
     NASSAU
     GREENE
     WESTCHESTER
     WESTCHESTER
     ERIE
     SENECA
     OTSEGO
     STEUBEN
     ORANGE
     ERIE
     ERIE
     MADISON
     ULSTER
     LIVINGSTON
     ALLEGANY
     DUTCHESS
     NIAGARA
     SCHUYLER
     ORANGE
     ALBANY
     CAYUGA
     COLUMBIA
     TIOGA
     KINGS
     YATES
     ALLEGANY
     ULSTER
     ERIE
     ONONDAGA
     WESTCHESTER
     ONEIDA
     ERIE
     SENECA
     ONEIDA
     BROOME
     NASSAU
     TOMPKINS
     PUTNAM
     WESTCHESTER
     STEUBEN
     MONROE
     ORANGE
     CATTARAUGUS
     SCHOHARIE
     CATTARAUGUS
     CAYUGA
     SCHUYLER
     MADISON
     DELAWARE
     CHAUTAUQUA
     WYOMING
     ONONDAGA
     CAYUGA
     RENSSELAER
     SCHOHARIE
     DUTCHESS
     SULLIVAN
     NIAGARA
     ONONDAGA
     ULSTER
     ORLEANS
     ALLEGANY
     GREENE
     SUFFOLK
     COLUMBIA
     WESTCHESTER
     WESTCHESTER
     NASSAU
     BROOME
     ORANGE
     HAMILTON
     ER

...

     CARBON
     SHERIDAN
     NATRONA
     SUBLETTE
     CONVERSE
     TETON
     WASHAKIE
     PLATTE
     CAMPBELL
     PLATTE
     LINCOLN
     LINCOLN
     UINTA
     GOSHEN
     PARK
     JOHNSON
     BIG HORN
     PARK
     BIG HORN
     ALBANY
     FREMONT
     LARAMIE
     ALBANY
     NIOBRARA
     CROOK
     CROOK
     SWEETWATER
     WESTON
OR
     MORROW
     HOOD RIVER
     TILLAMOOK
     LANE
     LANE
     JOSEPHINE
     CLACKAMAS
     MORROW
     WASCO
     DOUGLAS
     CLACKAMAS
     MALHEUR
     DESCHUTES
     JACKSON
     COOS
     YAMHILL
     JACKSON
     MARION
     CROOK
     LINCOLN
     MORROW
     MARION
     COLUMBIA
     CLATSOP
     UMATILLA
     CLACKAMAS
     WASHINGTON
     UNION
     BENTON
     MULTNOMAH
     WHEELER
     CLACKAMAS
     TILLAMOOK
     CLACKAMAS
     JACKSON
     GRANT
     LAKE
     JEFFERSON
     COOS
     BENTON
     JOSEPHINE
     CROOK
     MARION
     LINCOLN
     LANE
     GILLIAM
     COOS
     COLUMBIA
     UMATILLA
     KLAMAT

     MCPHERSON
     WOODSON
     CHAUTAUQUA
     NEOSHO
     DOUGLAS
CO
     EAGLE
     DELTA
     OTERO
     HINSDALE
     GARFIELD
     WELD
     ROUTT
     OURAY
     YUMA
     BOULDER
     ELBERT
     MOFFAT
     ARCHULETA
     EL PASO
     LOGAN
     PUEBLO
     WELD
     PARK
     SAN MIGUEL
     MONTROSE
     JEFFERSON
     GUNNISON
     LARIMER
     WELD
     BOULDER
     EAGLE
     DELTA
     SEDGWICK
     WELD
     WELD
     KIT CARSON
     PITKIN
     GARFIELD
     GILPIN
     ROUTT
     ARAPAHOE
     BOULDER
     MONTEZUMA
     PARK
     EL PASO
     ARCHULETA
     PUEBLO
     BACA
     LA PLATA
     WELD
     MONTROSE
     JEFFERSON
     MESA
     LAS ANIMAS
     GARFIELD
     MORGAN
     WELD
     OTERO
     WELD
     ADAMS
     GILPIN
     ARAPAHOE
     LA PLATA
     SUMMIT
     YUMA
     ALAMOSA
     LA PLATA
     BOULDER
     ARAPAHOE
     OURAY
     ELBERT
     EAGLE
     MESA
     LA PLATA
     EAGLE
     MONTEZUMA
     PUEBLO
     CHAFFEE
     EL PASO
     LARIMER
 

PA
     YORK
     CAMBRIA
     CAMBRIA
     MERCER
     WESTMORELAND
     ARMSTRONG
     BLAIR
     ADAMS
     LUZERNE
     LANCASTER
     BUCKS
     CHESTER
     BUTLER
     GREENE
     COLUMBIA
     SCHUYLKILL
     POTTER
     FAYETTE
     DELAWARE
     BEAVER
     ALLEGHENY
     ALLEGHENY
     LYCOMING
     WESTMORELAND
     LEBANON
     LAWRENCE
     CAMBRIA
     LACKAWANNA
     JEFFERSON
     TIOGA
     ARMSTRONG
     HUNTINGDON
     CLARION
     BLAIR
     WYOMING
     VENANGO
     ALLEGHENY
     LANCASTER
     CHESTER
     CARBON
     BERKS
     NORTHUMBERLAND
     LEHIGH
     ALLEGHENY
     WESTMORELAND
     WESTMORELAND
     YORK
     MERCER
     CHESTER
     ALLEGHENY
     WASHINGTON
     NORTHUMBERLAND
     PIKE
     LAWRENCE
     DELAWARE
     ALLEGHENY
     WASHINGTON
     LAWRENCE
     MCKEAN
     BUTLER
     SUSQUEHANNA
     FULTON
     BEAVER
     ALLEGHENY
     UNION
     YORK
     WESTMORELAND
     CAMBRIA
     LACKAWANNA
     FULTON
     ALLEGHENY
     MERCER
     WA

     DELAWARE
     NORTHAMPTON
     LYCOMING
     SUSQUEHANNA
     CHESTER
     LANCASTER
     LANCASTER
     BUCKS
     LUZERNE
     CUMBERLAND
     BERKS
     ALLEGHENY
     ALLEGHENY
     MONTGOMERY
     WESTMORELAND
     LAWRENCE
     TIOGA
     CENTRE
     WASHINGTON
     LACKAWANNA
     BRADFORD
     NORTHAMPTON
     HUNTINGDON
     BLAIR
     DELAWARE
     CAMBRIA
     FRANKLIN
     JEFFERSON
     CHESTER
     CLEARFIELD
     DELAWARE
     BUCKS
     SCHUYLKILL
     LANCASTER
     LANCASTER
     ALLEGHENY
     MONTGOMERY
     MONROE
     BERKS
     UNION
     SCHUYLKILL
     CAMBRIA
     FAYETTE
     LACKAWANNA
     CUMBERLAND
     MONTGOMERY
     CLINTON
     ARMSTRONG
     MONTGOMERY
     WAYNE
     BLAIR
     DELAWARE
     ADAMS
     LANCASTER
     BUCKS
     CAMBRIA
     BEAVER
     DAUPHIN
     ALLEGHENY
     SOMERSET
     CHESTER
     WESTMORELAND
     BERKS
     WESTMORELAND
     SCHUYLKILL
     WASHINGTON
     MONTGOMERY
     LEBANON
     CAMBRIA
     FAYETTE
     MCKEAN

ConnectionError: HTTPSConnectionPool(host='api.usa.gov', port=443): Max retries exceeded with url: /crime/fbi/sapi//api/summarized/agencies/PA0541300/offenses?api_key=qJDHVz0Vl9eSWuvXnvQ9HxHQPNeVRYdw08xTlUfw (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x127ec3128>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [7]:
# crimedata['results']
# 

In [8]:
# agencies = pd.read_csv('./data/agencies.csv')
# # agencies.head()
# print(agencies.loc[agencies['agency_id']==61][['ncic_agency_name','agency_type_name', 'population']])
# agencies.columns
# agencies_rev=agencies[['agency_id','county_name']]
# agencies_rev

In [9]:
# incidents = pd.read_csv('./data/NIBRS_incident.csv')
# incidents.head()
# incidents.dtypes
# incidents_rev = incidents[['agency_id','incident_date']]
# incidents_rev.dropna(axis=0, how='any',inplace=True)
# incidents_rev['AGENCY_ID'] =incidents_rev['agency_id'].astype('int64')
# incidents_rev.dtypes

In [10]:
# merge_data = incidents_rev.merge(agencies_rev, left_on='AGENCY_ID', right_on='agency_id', how='inner')
# merge_data

In [11]:
cdata = pd.DataFrame(mydata)
cdata.head()
crimegb = cdata.groupby(['Year','county','state']).sum()
crimegb.reset_index(inplace=True)
crimegb


,Year,county,state,count
0,2010,ABBEVILLE,SC,1402
1,2010,ADAIR,KY,137
2,2010,ADAIR,OK,839
3,2010,ADAMS,CO,19408
4,2010,ADAMS,NE,1797
5,2010,ADAMS,OH,571
6,2010,ADAMS,PA,1227
7,2010,ADAMS,WA,1858
8,2010,ADAMS,WI,982
9,2010,ADDISON,VT,1582


In [12]:
crimegb.to_csv('Crimedata.csv')